# Bayesian Inference in FSharp


## Introduction

For my F# Advent submission this year, I decided to implement an extremely simple Bayesian Statistical Framework to be used for inference. The purpose of statistical inference is to infer characteristics of a population via samples to be used for further tasks such as making predictions. The "Bayesian" prefix refers to making use of the Bayes Theorem to conduct inferential statistics.

![image.png](./Images/BayesTheoremNeon.jpeg)

I like to think of Statistical Inference via a pasta cooking analogy: hypothetically, assume you lose the cooking instructions of dried pasta you are trying to cook. You start boiling the water and don't know when the pasta is al-dente. One way to check is to careful remove a single piece and test if it is cooked to your liking. Through this process, you'll know right away if the pasta needs to cook more or is ready to be sauce'd up. Similarly, inference deals with trying to figure out characteristics (__is the pasta ready?__) of a population (__all the pasta__) via a sample (__a single piece of pasta__).

![image.png](./Images/pasta.jpg)

F#, as a language, didn't fail to deliver an awesome development experience! The particular aspects of the language that made it easy to develop a Bayesian Statistical Framework are [Pattern Matching](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/pattern-matching) and Immutable Functional Data Structures such as [Records](https://fsharpforfunandprofit.com/posts/records/) and [Discriminated Unions](https://fsharpforfunandprofit.com/posts/discriminated-unions/) that make expressing the domain succinctly and lucidly not only for the developer but also the reader.


## Motivation

I recently received a Masters in Data Science but discovered that none of my courses dove deep into Bayesian Statistics. Therefore, motivated by a penchant desire to fully understand the differences between frequentist approaches and Bayesian ones to eventually gain an understanding of Bayesian Neural Networks was the impetus behind trying to understand the statistical theory. 

While reading  ["A First Course in Bayesian Statistical Methods"](https://www.amazon.com/Bayesian-Statistical-Methods-Springer-Statistics/dp/1441928286/ref=sr_1_2?crid=3D7COZ06U9AU1&dchild=1&keywords=a+first+course+in+bayesian+statistical+methods&qid=1608705310&sprefix=A+first+course+in+baye%2Caps%2C239&sr=8-2) by Peter Hoff (highly recommend this book to those with a strong mathematical background), I discovered [pymc3](https://docs.pymc.io/), a Probabilistic Programming library in Python. Pymc3, in my opinion, has some great abstractions that I wanted to implement myself in a functional-first setting. 

The best way to learn a new statistical algorithm is my implementing it and the best way to learn something well is to teach it to others and hence, this submission is a result of that ideology. 

## Bayes Theorem

The crux of Bayesian Inference lies in making use of the Bayes Theorem whose formula is the following:

\begin{align}
p(\theta \mid y) = \frac{p(y \mid \theta) p(\theta)}{p(y)}
\end{align}

| Term              | Mathematical Representation | Definition  |
| :-------------    | :-------------------------: | ----------: |
| Variable to Infer |  $\theta$                   | The variable we are trying to infer from a population.        |
| Observations      |  $y$                        | Observations from a sampling model to validate our hypothesis.|
| Prior             |  $p(\theta)$                | The uncertainty of the hypothesis assocciated with the variable we are trying to infer __before__ any observations|  |
| Likelihood     | $p(y \mid \theta)$ | Sampling Model: Given our prior, the uncertainty of observing the data we have.  | | |
| Evidence       | $p(y)$ |  |
| Posterior      | $p(\theta \mid y)$ | The uncertanity associated with the variable we |

The interpretation of Bayes Theorem that resonates most with the [Scientific Method](https://en.wikipedia.org/wiki/Scientific_method) is that we are trying to quantify the validity of a hypothesis in light of new information. As an important aside, the type of the output i.e. the type of the posterior is a __probability distribution__ rather than a __point-estimate__.

## Bayesian Inference in Practice

In practice, the denominator or the evidence i.e. the normalization constant of the Bayes theorem is a pain to calculate. For example, for continuous variables, Bayes theorem has the following form application of the [Law of Total Probability](https://corporatefinanceinstitute.com/resources/knowledge/other/total-probability-rule/):

\begin{align}
p(\theta \mid y) = \frac{p(y \mid \theta) p(\theta)}{\int_{\Theta}p(y \mid \tilde{\theta}) p(\tilde{\theta}) \; d\tilde{\theta}}
\end{align}

That integral in the denominator is UGLY and sometimes computational infeasible and so, approximation techniques must be used to get to generate the posterior distribution : enter __Markov-Chain Monte Carlo (MCMC)__ methods taking advantage of the fact that the denominator is a normalization __constant__ with respect to the posterior or in other words:

\begin{align}
p(\theta \mid y) \propto p(y \mid \theta) p(\theta)
\end{align}

### Markov-Chain Monte-Carlo (MCMC) Algorithms

Let's break up the terms here for clarity:

####  Monte-Carlo

This term refers to fact that we make use of random numbers via a random number generator to approximate the posterior distribution to some reasonable degree of precision. The premise for the use of the Monte-Carlo method is that of the [Law of Large Numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers), which states that the average of the results of an experiment of a large number of iterations or trials tends to get closer to the expected value. 

The use case here is the large number of iterations guided by random number generators to cleverly approximate the posterior distribution will eventually resemble the posterior distribution.

#### Markov-Chain

"The future only depends on the present, not the past" (knicked this from Peter Hoff's AWESOME book mentioned before) encapsulates the idea behind Markov-Chains. Markov-Chains are a sequence of events whose probability is only a function of the previous event. To a computer scientist, a Markov-Chain can be thought of as a linked list (Thanks Steve!) of approximations whose values via Monte-Carlo methods typify the posterior.

#### Metropolis Hastings Algorithm

The Metropolis Hastings Algorithm is one choice of a MCMC Algorithm that can be used to approximate the posterior distribution. This algorithm, for each chain in the Markov-Chains, decides which proposal or step made in parameter space guided by randomness to accept or reject as a candidate for the next chain. The specific flavor of Metropolis Hastings I make use of is the Symmetric Metropolis Hastings algorithm and I'll elucidate on its implementation later on.

## Goals

The goals of this submission are to develop the following:

1. __A Bayesian Domain Specific Language (DSL) and Its Parser__ that a statistician with no programming experience can understand and use to specify a model and its parameters and code that parses the model.
2. __A Bayesian Network__ from the Parsed Bayesian DSL addressing the simplest case i.e. one random variable representing the prior and one for the likelihood.
3. __Symmeteric Metropolis-Hastings__, an MCMC algorithm that makes use of the Bayesian DSL representation to approximate the posterior distribution.

![image](./Images/Plan.png)

## Getting Setup

The inferential logic will require minimal dependencies. Here are the dependencies and their respective uses:

| Dependency        | Use          |
| :-----------------| :----------: | 
| __XPlot.Plotly__  | Used for charting the histogram of the distributions.
| __MathNet.Numerics__     | Used for the out-of-the-box statistical distributions. |
| __Newtonsoft.Json__      | Parsing the parameters of the Bayesian DSL. |

I wrote this notebook via the .net-fsharp kernel via .NET Interactive that's nothing short of awesome! This [blogpost](https://devblogs.microsoft.com/dotnet/net-core-with-juypter-notebooks-is-here-preview-1/) was pretty helpful to get me started.

In [1]:
// Dependencies
#r "nuget: XPlot.Plotly"
#r "nuget: MathNet.Numerics"
#r "nuget: Newtonsoft.Json"

// Imports 
open XPlot.Plotly
open MathNet.Numerics
open Newtonsoft.Json

Installing package XPlot.Plotly.done!

Installing package MathNet.Numerics.done!

Installing package Newtonsoft.Json.done!

## A Bayesian Domain Specific Language (DSL)  and Its Parser

![plan1](./Images/Plan_Section1.png)

The goal of this section is to develop "A Bayesian Domain Specific Language (DSL) that a statistician with no programming experience can understand and use to specify a model and its parameters and code that parses the model.". 

Drawing inspiration from [Stan](https://mc-stan.org) and pymc3 tutorials such as this [one](https://docs.pymc.io/notebooks/stochastic_volatility.html#Stochastic-Volatility-model), I wanted this DSL to be extremely simple but complete and therefore, the components I found that describe each random variable of the Bayesian Model are:

1. The Name of the Random Variable 
2. Conditionals of the Random Variables i.e. what other variables are given to be true to completely specify the random variable
3. The distribution associated with the random variable.
4. The parameters as variables of the distribution e.g. for a Normal Distribution, the parameters will be $\mu$ and $\sigma$, the mean and the variance respectively. 
5. The observed data for the Random Variable - if the random variable has 
6. A map of the parameters to constants to decouple the model from the values associated with the model.

### Format

```
Random Variable Name [|Comma-separated Conditionals] ~ Distribution(Comma-separated Parameters without spaces) [: observed] 
```

The details enclosed in ``[]`` imply optionality.

### Example

```
θ ~ Gamma(a,b)
Y|θ ~ Poisson(θ) : observed1
Z|Y,θ ~ Beta(θ, Y) : observed2
```

### The Domain

The domain or the types associated with the representation of the ``ParsedRandomVariable`` representing a single line in the specified model and the ``ParsedBayesianModel`` is a list of these.

In [2]:
type ParsedRandomVariable = 
    { Name             : string
      Conditionals     : string list
      Distribution     : string
      Parameters       : string list
      Observed         : string option }

type ParsedBayesianModel = ParsedRandomVariable list

### Parsing Logic

The parsing logic for each line in the user specified model is as follows:
1. Split the line by spaces.
2. Extract the Name and the Conditionals from the first part of the model split on the tilde (~). ``Random Variable Name | [Conditionals]``.
3. From the second part of the model split on the tilde, get the name of the distribution and its associated parameters along with the optionally available observed variable.

F#'s awesome pattern matching was a lifesaver here not only for it's ease of use but it's automatic ability to specify the failure cases. 

In [3]:
open System

// Format: RVName [|Conditionals] ~ Distribution(Parameters) [: observed] 
// [] -> optional
// NOTE: There can't be any spaces in the distribution parameters.
let parseLineOfModel (lineInModel : string) : ParsedRandomVariable = 
 
    // Helper fn to split the string based on a variety of type of delimiters.
    // Resultant type is a list of strings to feed in for the pattern matching.
    let splitToList (toSplit : string) (delimiters : obj) : string list = 
        let split = 
            match delimiters with
            | :? string        as s   -> toSplit.Split(s, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<string> as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<char>   as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | _ -> failwithf "Splitting based on delimiters failed as it is neither a string nor an array of strings: Of Type: %A - %A" (delimiters.GetType()) toSplit
        
        Array.toList split

    match splitToList lineInModel " " with
    | nameAndConditionals :: "~" :: distributionParametersObserved ->
        // Get the name and conditionals.
        let splitNameAndConditionals = splitToList nameAndConditionals "|"
        let name = splitNameAndConditionals.[0]
        let conditionals = 
            match splitNameAndConditionals with 
            | name :: conditionals -> 
                if conditionals.Length > 0 then splitToList conditionals.[0] ","
                else []
            | _ -> failwithf "Pattern not found for RV Name and Conditionals - the format is: RVName|Condtionals: %A" splitNameAndConditionals

        let extractAndGetParameters (distributionNameAndParameters : string) : string * string list = 
            let splitDistributionAndParameters = splitToList distributionNameAndParameters [| "("; ")" |]
            (splitDistributionAndParameters.[0], splitToList splitDistributionAndParameters.[1] ",")
            
        match distributionParametersObserved with 

        // Case: Without Observations. Example: θ ~ Gamma(a,b)
        | distributionNameAndParameters when distributionNameAndParameters.Length = 1 ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters.[0]
            { Name             = name; 
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = None; 
              Parameters       = snd extractedDistributionAndParameters; }

        // Case: With Observations. Example: Y|θ ~ Poisson(θ) : observed
        | distributionNameAndParameters :: ":" :: observed ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters
            { Name             = name;
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = Some observed.Head; // Only 1 observed list permitted.
              Parameters       = snd extractedDistributionAndParameters; } 

        // Case: Error.
        | _ -> failwithf "Pattern not found for the model while parsing the distribution, parameters and optionally, the observed variables: %A" distributionParametersObserved 

    | _ -> failwithf "Pattern not found for the following line in the model - please check the syntax: %A" lineInModel

let parseModel (model : string) : ParsedBayesianModel = 
    model.Split('\n') 
    |> Array.map(parseLineOfModel)
    |> Array.toList

// Helper to print out the Parsed Bayesian Model
let printParsedModel (model : string) : unit = 
    let parsedModel = parseModel model
    printfn "Model: %A is represented as %A" model parsedModel

#### Examples of Parsing a User-Specified DSL

In [4]:
// Print out our simple 1-Parameter Model.
let model1 = @"θ ~ Gamma(a,b)
              Y|θ ~ Poisson(θ) : observed"
printParsedModel(model1)

// This model doesn't make sense but adding to test multiple conditionals.
let model2  = @"θ ~ Beta(unit,unit)
               γ ~ Gamma(a,b)
               Y|θ,γ ~ Binomial(n,θ) : observed"
printParsedModel(model2)

Model: "θ ~ Gamma(a,b)
              Y|θ ~ Poisson(θ) : observed" is represented as [{ Name = "θ"
   Conditionals = []
   Distribution = "gamma"
   Parameters = ["a"; "b"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["θ"]
                        Distribution = "poisson"
                        Parameters = ["θ"]
                        Observed = Some "observed" }]
Model: "θ ~ Beta(unit,unit)
               γ ~ Gamma(a,b)
               Y|θ,γ ~ Binomial(n,θ) : observed" is represented as [{ Name = "θ"
   Conditionals = []
   Distribution = "beta"
   Parameters = ["unit"; "unit"]
   Observed = None }; { Name = "γ"
                        Conditionals = []
                        Distribution = "gamma"
                        Parameters = ["a"; "b"]
                        Observed = None }; { Name = "Y"
                                             Conditionals = ["θ"; "γ"]
                                             Distribution = "binomial"
              

### Specifying the Parameters

The idea is to decouple the parameters separate from the model and this is done by saving the details as a JSON string. This task is made simple using the ``Newtonsoft.Json`` library.

In [5]:
open System
open System.Collections.Generic

open MathNet.Numerics.Distributions

open Newtonsoft.Json

type Observed = float list

type ParameterList = 
    { Observed : float list; Parameters : Dictionary<string, float> } 

let deserializeParameters (paramsAsString : string) : ParameterList = 
    JsonConvert.DeserializeObject<ParameterList>(paramsAsString)

#### Examples of the Deserialization of the Parameters

In [6]:
// Parameter List 1
let parameters1 = "{Parameters : {μ0 : 0, σ0 : 1, μ : 5, σ : 2, λ : 4}, observed : [4.2,0.235,2.11]}"
let deserializedParameters1 = deserializeParameters parameters1
printfn "Deserialized Parameters 1: %A" (deserializedParameters1)

// Parameter List 2
let parameters2 = "{Parameters: {λ : 2}}"
let deserializedParameters2 = deserializeParameters parameters2
// Applying the Deserialized Parameters to Sample from a Distribution
let exp = Exponential deserializedParameters2.Parameters.["λ"] 
printfn "Sampling from the Exponential Distribution with the λ = %A parameter: %A" exp (exp.Sample())

Deserialized Parameters 1: { Observed = [4.2; 0.235; 2.11]
  Parameters = seq [[μ0, 0]; [σ0, 1]; [μ, 5]; [σ, 2]; ...] }
Sampling from the Exponential Distribution with the λ = Exponential(λ = 2) parameter: 0.4283604474


# Converting the Parsed Model into a Bayesian Network

![plan_section2](./Images/Plan_Section2.png)

The goal is to take the parsed representations of the user-inputted model to convert to a directed acylic graph that represents a Bayesian Network in an effort to easily and generically compute the posterior distribution. 

The first step here is to map the distribution that's specified by the user as aa string to an actual function that can compute the density or mass functions used in the prior and likelihood calculations. Subsequent to the distribution mapping, creating the notion of a Bayesian Node and the associated Bayesian Network will lead us to having a digestible form of the specified model. In this flavor of implementation, I will be simplifying the logic to only consider a two node network with one node representing the prior and the other, the likelihood. In a future post, I'll be generalizing this algorithm further.

### Distribution Mapping

The distribution mapping step involves developing:

1. Type-specified domain for the distribution.
2. Parsing logic that takes the name of the distribution and applies the ParsedRandomVariable based parameters to it.
3. A helper function based on the type of distribution to get the probability density function based on an input.

__NOTE: The list of distributions is incomplete, however, the idea should be similar to add newer ones.__

In [7]:
open MathNet.Numerics.Distributions

type DistributionType = 
    | Continuous
    | Discrete

type Parameter     = float
type DiscreteInput = int
type Input         = float

type DensityType = 
    | OneParameter         of (Parameter * Input -> float) 
    | OneParameterDiscrete of (Parameter * DiscreteInput -> float)
    | TwoParameter         of (Parameter * Parameter * Input -> float)

type DistributionInfo = { RVName             : string
                          DistributionType   : DistributionType 
                          DistributionName   : string 
                          Parameters         : float list
                          Density            : DensityType } with
                          
    static member Create (item : ParsedRandomVariable) 
                         (parameterList : ParameterList) : DistributionInfo = 
    // I know this is ugly but this functionality assumes the user enters the 
    // parameters in the order that's expected by the MathNet Numerics Library. 
    // Grab the parameters associated with this Random Variable.
    let rvParameters = 
        item.Parameters
        |> List.filter(parameterList.Parameters.ContainsKey) 
        |> List.map(fun item -> parameterList.Parameters.[item])

    // Extract Distribution Associated with the Parsed Random Variable.
    match item.Distribution with

    // 1 Parameter Distributions 
    | "exponential" -> 
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = OneParameter Exponential.PDF }
    | "poisson" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Discrete
          Density          = OneParameterDiscrete Poisson.PMF } 
    // 2 Parameter Distributions 
    | "normal" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = TwoParameter Normal.PDF }
    | "gamma" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = TwoParameter Gamma.PDF }
    | "beta" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = TwoParameter Beta.PDF }
    | "continuousuniform" ->
        { RVName           = item.Name
          DistributionName = item.Distribution
          Parameters       = rvParameters
          DistributionType = DistributionType.Continuous
          Density          = TwoParameter ContinuousUniform.PDF }
    // Failure Case
    | _ -> failwithf "Distribution not registered: %A" item.Distribution                      
                          
    member this.ComputeOneParameterPDF (parameter : float) (input : float) : float =
        match this.Density with
        | OneParameter pdf -> pdf(parameter,input)
        | _ -> failwithf "Incorrect usage of function with a non One Parameter Density Type. Density given: %A" this.Density 
    member this.ComputeOneParameterDiscretePMF (parameter : float) (input : int) : float =
        match this.Density with
        | OneParameterDiscrete pmf -> pmf(parameter,input)
        | _ -> failwithf "Incorrect usage of function with a non One Parameter Discrete Density Type. Density given: %A" this.Density 
    member this.ComputeTwoParameterPDF (parameter1 : float) (parameter2 : float) (input : float) : float =
        match this.Density with
        | TwoParameter pdf -> pdf(parameter1, parameter2, input)
        | _ -> failwithf "Incorrect usage of function with a non Two Parameter Density Type. Density given: %A" this.Density

### Helper Methods to get the Distributions from the User Specified DSL

In [8]:
let getDistributionInfoForModel(model : string) (parameterList : string) : DistributionInfo list = 
    let parsedModel   = parseModel model
    let parameterList = deserializeParameters parameterList 
    parsedModel
    |> List.map(fun x -> DistributionInfo.Create x parameterList)

let getDensityOrProbabilityForModel (model : string) 
                                    (parameterList : string) 
                                    (data : float seq) : IDictionary<string, float seq> = 
    getDistributionInfoForModel model parameterList
    |> List.map(fun (e : DistributionInfo) -> 
        match e.Density with
        | OneParameter p ->
            let param   = List.exactlyOne e.Parameters 
            let results = data |> Seq.map(fun d -> e.ComputeOneParameterPDF param d)
            e.RVName, results
        | OneParameterDiscrete p ->
            let param   = List.exactlyOne e.Parameters 
            let results = data |> Seq.map(fun d -> e.ComputeOneParameterDiscretePMF param (int d))
            e.RVName, results
        | TwoParameter p ->
            let p2 : float list = e.Parameters |> List.take 2
            let results = data |> Seq.map(fun d -> e.ComputeTwoParameterPDF p2.[0] p2.[1] d)
            e.RVName, results)
    |> dict

#### Testing the Distribution Mapping Logic

In [9]:
// Exponential. 
let exponentialModel     = "x ~ Exponential(lambda)"
let exponentialParamList = "{Parameters: {lambda : 2, a : 2., b : 2.3}, Observed : []}"
let exponentialDummyData = ContinuousUniform.Samples(0., 200.) |> Seq.take 2000
let exponentialPdfs      = getDensityOrProbabilityForModel exponentialModel exponentialParamList exponentialDummyData
printfn "Exponential: %A" (exponentialPdfs.Values)

// Normal.
let normalModel     = "x ~ Normal(mu,sigma)"
let normalParamList = "{Parameters: {mu: 0., sigma : 1.}, Observed : []}"
let normalDummyData = Normal.Samples(0.0, 1.0) |> Seq.take 2000
let normalPdfs      = getDensityOrProbabilityForModel normalModel normalParamList normalDummyData 
printfn "Normal: %A" (normalPdfs.Values)

// Poisson.
let poissonModel     = "x ~ Poisson(theta)"
let poissonParamList = "{Parameters: {theta: 44}, Observed : []}"
let poissonDummyData = ContinuousUniform.Samples(0., 5.) |> Seq.take 2000 
let poissonPdfs      = getDensityOrProbabilityForModel poissonModel poissonParamList poissonDummyData 
printfn "Poisson: %A" (poissonPdfs.Values)

Exponential: seq
  [seq
     [9.72139036e-174; 2.843236381e-64; 7.299313376e-37; 1.003053321e-113; ...]]
Normal: seq [seq [0.3447497275; 0.3818906797; 0.3740620767; 0.3027226526; ...]]
Poisson: seq
  [seq [7.781132241e-20; 7.781132241e-20; 7.532136009e-17; 3.423698186e-18; ...]]


### Bayesian Node

The next step is to specify the Bayesian Node, the abstraction that should encapsulate all the information for successfully computing probabilities. There can be two types of Nodes:

1. With Observed Variables for cases such as the Likelihood where we have data from a sampling model.
2. Non-Observed Variables for cases such as the Prior where all we have is a guess as to what the distribution of the parameter we want to infer is.

In [10]:
type BayesianNodeTypeInfo =
    | Observed of float list 
    | NonObserved

type BayesianNode = 
    { Name                 : string
      NodeType             : BayesianNodeTypeInfo 
      DistributionInfo     : DistributionInfo
      ParsedRandomVariable : ParsedRandomVariable } with

    static member Create(parsedRandomVariable : ParsedRandomVariable)
                        (parameterList : ParameterList) =

        let nodeType : BayesianNodeTypeInfo =
            match parsedRandomVariable.Observed with
            | Some _ -> BayesianNodeTypeInfo.Observed parameterList.Observed
            | None   -> BayesianNodeTypeInfo.NonObserved
        { Name                 = parsedRandomVariable.Name;
          NodeType             = nodeType;
          DistributionInfo     = DistributionInfo.Create parsedRandomVariable parameterList;
          ParsedRandomVariable = parsedRandomVariable; }

    member this.GetDependents (parsedBayesianModel : ParsedBayesianModel) : ParsedRandomVariable list =
        parsedBayesianModel 
        |> List.filter(fun x -> x.Conditionals |> List.contains(this.Name))

#### An Example of Creating a Node From a Line in the Model

In [11]:
let lineOfModel = @"x ~ Exponential(lambda) : observed"
let paramList   = "{Parameters: {lambda : 2, a : 2., b : 2.3 }, observed : [1,2,3,55]}"
BayesianNode.Create (parseLineOfModel lineOfModel) (deserializeParameters paramList)

Name,NodeType,DistributionInfo,ParsedRandomVariable
x,"{ FSI_0014+BayesianNodeTypeInfo+Observed: Item: [ 1, 2, 3, 55 ], Tag: 0, IsObserved: True, IsNonObserved: False }","{ FSI_0011+DistributionInfo: RVName: x, DistributionType: { FSI_0011+DistributionType: Tag: 0, IsContinuous: True, IsDiscrete: False }, DistributionName: exponential, Parameters: [ 2 ], Density: { FSI_0011+DensityType+OneParameter: Item: { FSI_0011+Create@41: }, Tag: 0, IsOneParameter: True, IsOneParameterDiscrete: False, IsTwoParameter: False } }","{ FSI_0006+ParsedRandomVariable: Name: x, Conditionals: [ ], Distribution: exponential, Parameters: [ lambda ], Observed: { Microsoft.FSharp.Core.FSharpOption<System.String>: Value: observed } }"


### The Simple Bayesian Network

A General Bayesian Network is a Directed Acyclic Graph consisting of Bayesian Nodes and the relationship amongst them. The Simple Bayesian Network is a type of Bayesian Network with two Bayesian Nodes namely, the Prior and Likelihood nodes or in other words, the model that the user can specify is constrained to two random variables; this is done for the sake of simplicity.

The Simple Bayesian Network also consists of functions that can be used to compute the numerator of Bayes Theorem i.e. $Posterior = Prior \times Likelihood$

The __Prior__ is computed using the PDF or PMF based on the parameters associated with the Prior Bayesian Node. For example, a Normal Prior Node will have the formula of ``Normal.PDF(mean, stddev, input)``. 

The __Likelihood__, on the other hand, involves the result from the Prior and the observed list. The assumptions in the computation of the Likelihood is that the observations are __conditionally independent__ given our prior and the __exchangability__ of the observations where the order of the observations don't matter i.e. the order or labels of the observations don't change the outcome.

\begin{align}
p(y_1, \dots, y_n \mid \theta) = p(y_1 \mid \theta) \times \dots \times p(y_n \mid \theta) = \prod_i p(y_i \mid \theta).
\end{align}

In other words, we will be computing the PDF based on the prior, parameters of the likelihood node and the observed variables. For example, if we were to be inferring the mean of the Normal distribution, our likelihood will be computed by the following pseudo-code: ``product(Normal.PDF(prior, stddevOfLikelihood, observation))``


In [12]:
// Only to be used for a model with 2 nodes 
// i.e. one for the prior and one for the likelihood.
type SimpleBayesianNetworkModel = 
  { Name       : string
    Nodes      : IDictionary<string, BayesianNode>
    Prior      : BayesianNode
    Likelihood : BayesianNode } with

    member this.GetPriorProbability (input : float) : float = 
      let distributionInfo = this.Prior.DistributionInfo 
      match distributionInfo.Density with
      | OneParameter p ->
          let param = List.exactlyOne distributionInfo.Parameters 
          distributionInfo.ComputeOneParameterPDF param input 
      | OneParameterDiscrete p ->
          let param   = List.exactlyOne distributionInfo.Parameters 
          distributionInfo.ComputeOneParameterDiscretePMF param (int input)
      | TwoParameter p ->
          let p2 : float list = distributionInfo.Parameters |> List.take 2
          distributionInfo.ComputeTwoParameterPDF p2.[0] p2.[1] input

    member this.GetLikelihoodProbability (prior : float) : float = 
        let distributionInfo = this.Likelihood.DistributionInfo
        let observed         = match this.Likelihood.NodeType with
                               | Observed l -> l
                               | _ -> failwithf "Incorrectly constructed Simple Network Model. %A" this 
        let density = 
          match distributionInfo.Density with
          | OneParameter p -> 
              observed |> List.map(fun d -> distributionInfo.ComputeOneParameterPDF prior d)
          | OneParameterDiscrete p ->
              observed |> List.map(fun d -> distributionInfo.ComputeOneParameterDiscretePMF prior (int (Math.Ceiling d)))
          | TwoParameter p ->
              let p : float = distributionInfo.Parameters |> List.exactlyOne 
              observed |> List.map(fun d -> distributionInfo.ComputeTwoParameterPDF prior p d)
        density
        |> List.fold (*) 1.0

    member this.GetPosteriorWithoutScalingFactor (input: float) : float = 
      let priorPdf      = this.GetPriorProbability input 
      let likelihoodPdf = this.GetLikelihoodProbability priorPdf
      priorPdf * likelihoodPdf

    static member Construct (name : string)
                            (model : ParsedBayesianModel)
                            (parameterList : ParameterList) : SimpleBayesianNetworkModel = 

        // Construct all the modes of the model.
        let allNodes : (string * BayesianNode) list =
            model
            |> List.map(fun m -> m.Name, BayesianNode.Create m parameterList)

        let prior : BayesianNode =  
            allNodes
            |> List.filter(fun (_,m) -> m.NodeType = BayesianNodeTypeInfo.NonObserved) 
            |> List.map(fun (_,m) -> m)
            |> List.exactlyOne

        let likelihood : BayesianNode =  
            allNodes
            |> List.filter(fun (_,m) -> m.NodeType <> BayesianNodeTypeInfo.NonObserved)
            |> List.map(fun (_,m) -> m)
            |> List.exactlyOne

        { Name       = name; 
          Nodes      = dict allNodes; 
          Prior      = prior;
          Likelihood = likelihood; } 

#### Example of Creating a Simple Bayesian Network

In [13]:
let model = @"x ~ Normal(μ,τ) 
              y|x ~ Normal(x,σ) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {μ : 5, τ : 3.1622, σ : 1}, observed : [9.37,10.18,9.16,11.60,10.33]}"
let normalModel = 
    SimpleBayesianNetworkModel.Construct "Normal-Normal" parsedModel (deserializeParameters paramList)
normalModel

Name,Nodes,Prior,Likelihood
Normal-Normal,"[ { System.Collections.Generic.KeyValuePair<System.String,FSI_0014+BayesianNode>: Key: x, Value: { FSI_0014+BayesianNode: Name: x, NodeType: { FSI_0014+BayesianNodeTypeInfo+_NonObserved: Tag: 1, IsObserved: False, IsNonObserved: True }, DistributionInfo: { FSI_0011+DistributionInfo: RVName: x, DistributionType: { FSI_0011+DistributionType: Tag: 0, IsContinuous: True, IsDiscrete: False }, DistributionName: normal, Parameters: [ 5, 3.1622 ], Density: { FSI_0011+DensityType+TwoParameter: Item: FSI_0011+Create@54-2, Tag: 2, IsOneParameter: False, IsOneParameterDiscrete: False, IsTwoParameter: True } }, ParsedRandomVariable: { FSI_0006+ParsedRandomVariable: Name: x, Conditionals: [ ], Distribution: normal, Parameters: [ μ, τ ], Observed: <null> } } }, { System.Collections.Generic.KeyValuePair<System.String,FSI_0014+BayesianNode>: Key: y, Value: { FSI_0014+BayesianNode: Name: y, NodeType: { FSI_0014+BayesianNodeTypeInfo+Observed: Item: [ 9.37, 10.18, 9.16, 11.6, 10.33 ], Tag: 0, IsObserved: True, IsNonObserved: False }, DistributionInfo: { FSI_0011+DistributionInfo: RVName: y, DistributionType: { FSI_0011+DistributionType: Tag: 0, IsContinuous: True, IsDiscrete: False }, DistributionName: normal, Parameters: [ 1 ], Density: { FSI_0011+DensityType+TwoParameter: Item: FSI_0011+Create@54-2, Tag: 2, IsOneParameter: False, IsOneParameterDiscrete: False, IsTwoParameter: True } }, ParsedRandomVariable: { FSI_0006+ParsedRandomVariable: Name: y, Conditionals: [ x ], Distribution: normal, Parameters: [ x, σ ], Observed: { Microsoft.FSharp.Core.FSharpOption`1[System.String]: Value: observed } } } } ]","{ FSI_0014+BayesianNode: Name: x, NodeType: { FSI_0014+BayesianNodeTypeInfo+_NonObserved: Tag: 1, IsObserved: False, IsNonObserved: True }, DistributionInfo: { FSI_0011+DistributionInfo: RVName: x, DistributionType: { FSI_0011+DistributionType: Tag: 0, IsContinuous: True, IsDiscrete: False }, DistributionName: normal, Parameters: [ 5, 3.1622 ], Density: { FSI_0011+DensityType+TwoParameter: Item: { FSI_0011+Create@54-2: }, Tag: 2, IsOneParameter: False, IsOneParameterDiscrete: False, IsTwoParameter: True } }, ParsedRandomVariable: { FSI_0006+ParsedRandomVariable: Name: x, Conditionals: [ ], Distribution: normal, Parameters: [ μ, τ ], Observed: <null> } }","{ FSI_0014+BayesianNode: Name: y, NodeType: { FSI_0014+BayesianNodeTypeInfo+Observed: Item: [ 9.37, 10.18, 9.16, 11.6, 10.33 ], Tag: 0, IsObserved: True, IsNonObserved: False }, DistributionInfo: { FSI_0011+DistributionInfo: RVName: y, DistributionType: { FSI_0011+DistributionType: Tag: 0, IsContinuous: True, IsDiscrete: False }, DistributionName: normal, Parameters: [ 1 ], Density: { FSI_0011+DensityType+TwoParameter: Item: { FSI_0011+Create@54-2: }, Tag: 2, IsOneParameter: False, IsOneParameterDiscrete: False, IsTwoParameter: True } }, ParsedRandomVariable: { FSI_0006+ParsedRandomVariable: Name: y, Conditionals: [ x ], Distribution: normal, Parameters: [ x, σ ], Observed: { Microsoft.FSharp.Core.FSharpOption<System.String>: Value: observed } } }"


#### Debugging the Prior and Likelihood

A quick sanity check to see if the numbers match up.

In [14]:
let model = @"x ~ Normal(μ,τ) 
              y|x ~ Normal(x,σ) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {μ : 5, τ : 3.1622, σ : 1}, observed : [9.37,10.18,9.16,11.60,10.33]}"
let normalModel = 
    SimpleBayesianNetworkModel.Construct "Normal-Normal" parsedModel (deserializeParameters paramList)
    
// Prior Check
let priorFromModel = normalModel.GetPriorProbability 0.0
let computedPrior  = Normal.PDF(5., 3.1622, 0.0)
printfn "Normal Model Prior: %A | Prior from formula: %A" priorFromModel computedPrior

// Likelihood Check
let likelihoodFromModel = normalModel.GetLikelihoodProbability 0.0
let observed : float list = 
    match normalModel.Likelihood.NodeType with
    | Observed x -> x
    | _ -> failwithf "Incorrectly constructed Likelihood"
    
let likelihoodListFromFormula : float list = 
    observed |> List.map(fun (o:float) -> Normal.PDF(computedPrior, 1., o))
let likelihoodFromFormula = likelihoodListFromFormula |> List.fold (*) 1.

printfn "Normal Model Likelihood: %A | Likelihood from formula: %A" 
    (normalModel.GetLikelihoodProbability 0.03614314702) 
    (likelihoodFromFormula)

Normal Model Prior: 0.03614314702 | Prior from formula: 0.03614314702
Normal Model Likelihood: 4.158402902e-114 | Likelihood from formula: 4.158402901e-114


# Applying the Symmetric Metropolis Hastings Algorithm to Approximate the Posterior Distribution

![plan_section3](./Images/Plan_Section3.png)

Now that we have some digestible representation of the Simple Bayesian Network, our next step is to make use of an MCMC algorithm to approximate the posterior. The steps here as follows:

1. Define a well-specified MCMC domain to run the algorithmss.
2. Implement a type of MCMC algorithm called the Symmetric Metropolis Hasting Algorithm.
3. Apply the algorithm to a Simple Bayesian Network.

### The Symmetric Metropolis Hastings Algorithm

I would consider the Symmetric Metropolis Hastings Algorithm to be a simple MCMC algorithm that's based on some smart applications of the Bayes Theorem and therefore, I chose it to be the "toy" approximation algorithm used here. There are better algorithms out there such as [Hamiltonian Monte-Carlo](http://diffsharp.github.io/DiffSharp/examples-hamiltonianmontecarlo.html) or the [No U-Turn Sampler](http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf)

#### The Algorithm
1. Produce a proposal based on the proposal distribution and the current value.
2. Compute the Posterior with the proposal and the current value.
3. Compute the Acceptance Ratio.
4. Accept or Reject the proposal. If accepted, the current value will be the proposed value else, grab another proposal using the proposal distribution.
5. Repeat until iterative convergence for all chains requested.

The main ideas associated with the steps or single iterations of this algorithm are:

#### 1. Proposing New Values To Approximate the Parameter Space

Using a proposal distribution, we come up with newer values of the parameter to explore and find regions that are more likely to be outputted by the posterior distribution. Examples of the proposal distributions are:

1. ``Uniform(current - δ, current + δ)``
2. ``Normal(current, δ^2)``

where δ is a hyperparameter that's a proxy of the size of the jumps.

#### 2. Computing the Acceptance Ratio

Taking advantage of the notion below and considering different parameters, say $\theta$ and $\theta^{*}$, we come up with a new quantity called the __Acceptance Ratio__, $r$.

$p(\theta \mid y) \propto p(y \mid \theta) p(\theta)$

Where $r =  \frac{p(\theta \mid y)}{p(\theta^{*} \mid y)} = \frac{p(y \mid \theta) p(\theta)}{p(y \mid \theta^{*}) p(\theta^{*})}$

By examining this ratio, it can be made evident that if the numerator is greater than the denominator, we have an acceptance ratio > 1; this implies that the posterior without the normalization constant with $\theta$ is higher than that with $\theta^{*}$ or is a data-point more likely (or with less uncertainity) to be obtained from the posterior.

#### 3. Accepting or Rejecting The Proposal Based on the Acceptance Ratio

The Symmetric Metropolis Hastings algorithm is a type of Metropolis Hastings algorithm where the proposal distribution between the current the proposed, new value is symmetric i.e. the probability to obtain the proposal given the current value is the same as the probability to obtain the current value given the proposal.

With this idea in mind, once we have computed the acceptance ratio, we want to check to see how likely it'll typify the posterior distribution. This can be done by taking a random drawing from a uniform distribution where the lower and upper values are 0 and 1 respectively. This idea can be elucidated on by the fact that if the proposal based posterior better approximates the real-posterior better than a random drawing from a 0-1 uniform distribution, in the long run, it'll eventually be close-enough to the real-posterior.

As mentioned before, if the proposal was accepted, we use that as the next current value to generate another proposal or else try to get another proposal in the subsequent iteration.

#### Weaknesses of this Algorithm

The simplicity of this model comes with costs, specifically, the dependence on the hyperparameters that have to be tuned and autocorrelation of values or if the proposed distribution is extremely close to the current value, it'll take longer to explore the entire parameter space as the jumps will be smaller. As an example of the weaknesses, I needed to experiment with a bunch of hyperparameters before I got to some semblance of the posterior distribution I was expecting in addition to increases the number of iterations.

### Defining the MCMC Domain

I liked the idea of using a simple ``Request -> Result`` model from the Client-Server model to encapulate the approximation. Additionally, I drew inspiration from reading pymc3's code to come up with the following domain.

In general, I want any of the prospective MCMC algorithms to have the ability to:

1. Output multiple chains that approximate the posterior.
2. Have the ability to specify multiple types of "steps" or mechanisms to induce jumps in parameter space for appropriately approximating the posterior distribution.
3. Outputting statistics as to how the algorithm did.
4. Specifying a converge criteria after which the MCMC algorithm stops the approximation.
5. An ability to pass in a proposal distribution on the basis of which jumps in the parameter space are induced.

#### Terminology 

The __Acceptance Rate__ is a result of an MCMC chain the proportion of the number of accepted proposals to the total number of non-burn in proposals. This metric is a proxy of how the steps are approximating the posterior.

The __BurnInIterationsPct__ is the percent of posterior approximates to ignore; these are taken from the beginning of the chains and typify the first few jumps before the algorithm is at a point where it's better at approximating the posterior.

In [15]:
type ConvergenceCriteria = 
    | IterativeConvergence of int // Number of Iterations

type ProposalDistribution =
    | Normal of float  // Normal( current, delta )
    | ContinuousUniform of float // ContinuousUniform( current - delta, current + delta )
    | PositiveContinuousUniform of float // ( x ~ Uniform( current - delta, current + delta ) if x <= 0 then 0.1 else x )

type MCMCInferenceStep = 
    | SymmetricMetropolisHastings of ProposalDistribution * float

type MCMCChain =
    { Id             : int
      AcceptanceRate : float
      StepValues     : float seq }

type MCMCRequest = 
    { StepFunction         : MCMCInferenceStep
      ConvergenceCriteria  : ConvergenceCriteria
      BurnInIterationsPct  : float 
      Chains               : int 
      PrintDebug           : bool }

type MCMCResult =
    { Chains         : MCMCChain seq 
      MCMCRequest    : MCMCRequest }

In [16]:
open System
open MathNet.Numerics.Distributions
open XPlot.Plotly

type AcceptanceRejection =
    | Acceptance of float // Acceptance of the Proposal
    | Rejection  of float // Rejection of the Proposal

let doSymmetricMetropolisHastings (request : MCMCRequest) 
                                  (iterations : int)
                                  (proposalDistribution : ProposalDistribution)
                                  (initialValue : float)
                                  (simpleBayesianModel : SimpleBayesianNetworkModel) : MCMCResult =

    let getChain (id : int) (request : MCMCRequest) 
                 (iterations : int) (simpleBayesianModel : SimpleBayesianNetworkModel) : MCMCChain =

        let burnin         = int (Math.Ceiling(request.BurnInIterationsPct / 100. * float iterations))
        let zeroOneUniform = ContinuousUniform()

        let mutable current = simpleBayesianModel.GetPosteriorWithoutScalingFactor initialValue

        let matchedProposalDistribution (input : float) :  float =
            match proposalDistribution with 
            | Normal delta            -> Normal(input, delta).Sample()
            | ContinuousUniform delta -> ContinuousUniform(input - delta, input + delta).Sample()
            | PositiveContinuousUniform delta -> 
                let u = ContinuousUniform(input - delta, input + delta).Sample()
                if u <= 0. then input + 0.1 else u

        let step (iteration : int) : AcceptanceRejection =
            let proposed           = matchedProposalDistribution current
            let currentProbability = simpleBayesianModel.GetPosteriorWithoutScalingFactor current
            let proposedProbabilty = simpleBayesianModel.GetPosteriorWithoutScalingFactor proposed
            let acceptanceRatio    = Math.Min(currentProbability / proposedProbabilty, 1.)
            let uniformDraw        = zeroOneUniform.Sample()

            if request.PrintDebug then 
                printfn "Chain: %A Iteration: %A - Current Probability: %A | Proposed Probability: %A | AcceptanceRatio: %A | Uniform Draw: %A | Current: %A | Proposed: %A" 
                    id iteration currentProbability proposedProbabilty acceptanceRatio uniformDraw current proposed

            if uniformDraw < acceptanceRatio then (current <- proposed; Acceptance proposed) 
            else Rejection current
    
        let stepResults : AcceptanceRejection seq =
            seq {1..iterations}
            |> Seq.map step
            |> Seq.skip burnin

        let getAcceptanceRateAndStepValues : float * float seq =

            // Compute the Acceptance Rate.
            let acceptanceRate : float = 
                let totalBurninWithoutBurnin : float = float (Seq.length stepResults)
                let totalNumberOfAcceptances : float = 
                    stepResults 
                    |> Seq.filter(fun x -> 
                        match x with
                        | Acceptance x -> true
                        | _ -> false)
                    |> Seq.length
                    |> float
                totalNumberOfAcceptances / totalBurninWithoutBurnin

            // Grab the Step Values that'll approximate the posterior.
            let stepValues =
                stepResults |> Seq.map(fun s ->
                    match s with
                    | Acceptance v -> v
                    | Rejection v -> v)

            acceptanceRate, stepValues

        let acceptanceRate, stepValues = getAcceptanceRateAndStepValues 

        { Id             = id
          AcceptanceRate = acceptanceRate
          StepValues     = stepValues} 

    let chains : MCMCChain seq =
        seq {1..request.Chains}
        |> Seq.map(fun id ->  getChain id request iterations simpleBayesianModel)

    { Chains      = chains
      MCMCRequest =  request }

### Running the Algorithm

This method takes in a generic MCMC Request and a model and returns an MCMC Result consisting of the chain of results that have the posterior sequence as well as the acceptance rate.

In [17]:
let runMCMC (request : MCMCRequest) 
            (model : SimpleBayesianNetworkModel) : MCMCResult =
    match request.StepFunction with
    | SymmetricMetropolisHastings (proposalDistribution,initialValue) ->
        match request.ConvergenceCriteria with 
        | IterativeConvergence iterations -> 
            doSymmetricMetropolisHastings request iterations proposalDistribution initialValue model
        | _ -> failwith "You need to pass in the number of iterations for the Metropolis-Hastings algorithm"
    | _ -> failwithf "Step Function Not Registered: %A" request.StepFunction

### Tieing it All Together

Let's now go from a user inputted model to the results of the Symmetric Metropolis Hastings algorithm to approximate the posterior in two cases. We'll plot the results of the first chain to highlight if the algorithm did the right thing.

#### Exponential Prior and Exponential Likelihood

In [18]:
let model = @"x ~ Exponential(a) 
              y|x ~ Exponential(x) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {a : 2., b : 2.}, observed : [1,2,3,4,4,2,5,6,7,3,2,3,4,5,6,1,2,3,4,4,4,4]}"
let simpleModel = SimpleBayesianNetworkModel.Construct "Exponential Model" parsedModel (deserializeParameters paramList)

let request : MCMCRequest = 
    { StepFunction        = SymmetricMetropolisHastings (ProposalDistribution.PositiveContinuousUniform 0.002, 0.001)
      ConvergenceCriteria = IterativeConvergence 10000
      BurnInIterationsPct = 0.2
      PrintDebug          = false 
      Chains              = 4 }

let mcmc       = runMCMC request simpleModel
let firstChain = Seq.head mcmc.Chains

Histogram(x = firstChain.StepValues)
|> Chart.Plot
|> Chart.WithTitle "Exponential Prior and Likelihood"

As the online notebook renderer isn't displaying the chart, I took a screenshot of it.

![Exponential Prior and Likelihood](./Images/ExponentialPriorLikelihood.png)

This histogram of the Step Values indicates that the Posterior has the shape of an Exponential distribution; which is expected and confirms.

#### Normal Prior and Exponential Likelihood

In [21]:
#!fsharp
let model = @"x ~ Normal(μ,τ) 
              y|x ~ Exponential(x) : observed"
let parsedModel = parseModel model
let paramList = "{Parameters: {μ : 5, τ : 3.1622, σ : 1}, observed : [9.37,10.18,9.16,11.60,10.33]}"
let simpleModel = 
    SimpleBayesianNetworkModel.Construct "Normal-Exponential" parsedModel (deserializeParameters paramList)


let request : MCMCRequest = 
    { StepFunction        = SymmetricMetropolisHastings (ProposalDistribution.PositiveContinuousUniform 0.003, 0.0001)
      ConvergenceCriteria = IterativeConvergence 50000
      BurnInIterationsPct = 0.2
      PrintDebug          = false 
      Chains              = 4 }

let mcmc       = runMCMC request simpleModel
let firstChain = Seq.head mcmc.Chains

Histogram(x = firstChain.StepValues)
|> Chart.Plot
|> Chart.WithTitle "Normal Prior with Exponential Likelihood"
|> Chart.WithWidth 700
|> Chart.WithHeight 500

This posterior distribution was a lot more variable in terms of the shape of the distribution; it was also extremely dependent on the hyperparameters passed in highlighting the weaknesses of this algorithm. I am less confident about these results than those from the previous example.

![Normal-Exponential](./Images/NormalExponential.png)

### How to Use the Posterior

Now that the posterior is obtained, the statistician could use it by sampling from it to plug in to other models used for prediction such as use it for the [Posterior Predictive Distribution](https://en.wikipedia.org/wiki/Posterior_predictive_distribution).

My plan is to write a separate blogpost to go over computing this and making use of the posterior to make predictions. This submission is already bloated enough.

## Conclusion

This submission took a lot of work and is a product of two months of going down a Bayesian rabbit hole. Since I think I have built a reasonable base to continue to build on top of. As an obvious disclaimer, none of this code should be used in production - some of it is untested and might have bugs that weren't apparent to me while writing this. 

Also, please let me know if you find any mistakes or can suggest any improvements! I'll be happy to learn from them and fix up this notebook.

Thanks to the organizers of FSharp-Advent and Happy Holidays to all!

# Lessons and Prospective Improvements

## Lessons

### Debugging an Iterative Algorithm Can Be Challenging

Despite being a simple algorithm, the Symmetric Metropolis Hastings still has a lot of moving parts. Debugging through this was a big challenge! As much of cliche as it is, starting off small and passing in extremely simple parameters to complex algorithms was the way I debugged it. Also, adding optional print statements were helpful.


### Tooling And Workflow is of Paramount Importance

Having the right workflow is extremely important for your productivity. I landed on making use of Jupyter notebooks with the .NET Interactive F# kernel. I briefly tried out the VSCode notebooks but got into issues such as inability to stop the kernel or cells running indefinitely. I will gladly assist with giving any feedback here as I think that feature has a lot of potential.

### Prototyping Helps With Creating Modular Functions

I took a different approach with this submission than my previous one by first working on prototypical functions. I created a separate notebook where I wrote up all the functions / defined my domain and essentially prototyped the design. This was helpful as it gave me an opportunity to write and debug the algorithms productively.

## Improvements

### Better Parameter Parsing and Application

I dislike the current parameter parsing logic when mapping to distributions. 

### More Generalized Bayesian Network

Right now, I have implemented a Simple Bayesian Network but want to generalize the implementation to a Generic Bayesian Network that'll work for all cases.

### More Efficient MCMC Methods

As noted before, the Symmetric Metropolis Hastings algorithm has its flaws and there are more complex algorithms that give better results out there. I want to implement these from scratch.

## Further Resources

Added to the Readme associated with this repository or [here](https://github.com/MokoSan/FSharpAdvent_2020/blob/main/README.md).

## Music Listened to While Working On This Project

1. [John Zorn and Friends](https://www.youtube.com/watch?v=c4eO2o9u1j0&ab_channel=podgoryt)
2. [Mike Patton and Mondo Cane](https://www.youtube.com/watch?v=iDOl5q7UXfg&ab_channel=FNM4EVER2)
3. [Burt Bacharach - This Guy's In Love with you](https://www.youtube.com/watch?v=2dDGnl8_Dzg&ab_channel=MusicWonders )
4. [Melvins - Lysol](https://www.youtube.com/watch?v=JtO_Awk4pqU&ab_channel=AboveDeath)
5. [The Jesus Lizard - Puss](https://www.youtube.com/watch?v=PaBJQG6A9SQ&ab_channel=spycory1)
6. [Mr. Bungle - Sudden Death](https://www.youtube.com/watch?v=-QWFV_057KM&ab_channel=IpecacRecordings)
7. [Madreblu - Certamente](https://www.youtube.com/watch?v=Ivh0FWTSJ78&ab_channel=Milano2000Records)
8. [Pato Banton - Spirits in the Material World](https://www.youtube.com/watch?v=S--kTyvm_fM&ab_channel=PatoBanton%26TheReggaeRevolution-Topic)
9. [Max Cooper - Ripple](https://www.youtube.com/watch?v=P_X1KGCgWlE&ab_channel=MaxCooper-Topic)
10. [The Sword - Cheap Sunglasses](https://www.youtube.com/watch?v=RYnTLIZkjlg&ab_channel=SwordofDoomMusic)
11. [Brother Dege - Too Old To Die Young](https://www.youtube.com/watch?v=FQNFcYvILJE&ab_channel=UltimatePowa)
12. [Mr. Bungle - Retrovertigo](https://www.youtube.com/watch?v=DRyh2cxJCp0&ab_channel=tkan)